# Defi-IA 2022 <a href="https://www.kaggle.com/c/defi-ia-2022/overview"><img src="https://upload.wikimedia.org/wikipedia/commons/7/7c/Kaggle_logo.png" style="max-width: 70px; display: inline" alt="Python"/></a>
-------------------------------------------------------------

**Predict the accumulated daily rainfall on ground stations**  
*Quentin Douzery, Alexia Ghozland, Dario Moed*

## <font color = 'royalblue'>1 Imports</font>

In [50]:
##Packages
import time
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.model_selection import GridSearchCV,train_test_split 
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor,MLPClassifier
from sklearn.ensemble import RandomForestClassifier ,RandomForestRegressor
from sklearn.preprocessing import scale , StandardScaler

In [2]:
##Données
coords_path = '/Users/qdouzery/Desktop/INSAT_Defi-IA-2022/Data/Other/stations_coordinates.csv' #Perso
#coords_path = '/home/douzery/Bureau/Defi-IA-2022_Data/Other/stations_coordinates.csv' #INSA
#coords_fname  = '/kaggle/input/defi-ia-2022-test/Other/Other/stations_coordinates.csv'

xtrainS_path = '/Users/qdouzery/Desktop/INSAT_Defi-IA-2022/Data/Train/X_station_train.csv' #Perso
#xtrainS_path = '/home/douzery/Bureau/Defi-IA-2022_Data/Train/X_station_train.csv' #INSA
#fname = '/kaggle/input/defi-ia-2022-test/Train/Train/X_station_train.csv'

ytrain_path = '/Users/qdouzery/Desktop/INSAT_Defi-IA-2022/Data/Train/Y_train.csv' #Perso
#ytrain_path = '/home/douzery/Bureau/Defi-IA-2022_Data/Train/Y_train.csv' #INSA
#fname = '/kaggle/input/defi-ia-2022-test/Train/Train/Y_train.csv'

xtestS_path = '/Users/qdouzery/Desktop/INSAT_Defi-IA-2022/Data/Test/X_station_test.csv' #Perso
#xtrainS_path = '/home/douzery/Bureau/Defi-IA-2022_Data/Train/X_station_train.csv' #INSA

bl_obsT_path = '/Users/qdouzery/Desktop/INSAT_Defi-IA-2022/Data/Test/Baselines/Baseline_observation_test.csv' #Perso

## <font color = 'royalblue'>2 Récupération des données</font>

In [3]:
##X_train
xtrainS = pd.read_csv(xtrainS_path,parse_dates=['date'],infer_datetime_format=True)
xtrainS['number_sta'] = xtrainS['number_sta'].astype('category')  
display(xtrainS)
print('Missing Data :', xtrainS.isna().sum().sum(), '\n', 'Shape is :', xtrainS.shape)

,number_sta,date,ff,t,td,hu,dd,precip,Id
0,14066001,2016-01-01 00:00:00,3.05,279.28,277.97,91.4,200.0,0.0,14066001_0_0
1,14066001,2016-01-01 01:00:00,2.57,278.76,277.45,91.4,190.0,0.0,14066001_0_1
2,14066001,2016-01-01 02:00:00,2.26,278.27,277.02,91.7,181.0,0.0,14066001_0_2
3,14066001,2016-01-01 03:00:00,2.62,277.98,276.95,93.0,159.0,0.0,14066001_0_3
4,14066001,2016-01-01 04:00:00,2.99,277.32,276.72,95.9,171.0,0.0,14066001_0_4
...,...,...,...,...,...,...,...,...,...
4409469,95690001,2017-12-30 19:00:00,9.10,286.68,283.44,80.8,239.0,0.0,95690001_729_19
4409470,95690001,2017-12-30 20:00:00,8.58,286.39,283.21,81.1,231.0,0.0,95690001_729_20
4409471,95690001,2017-12-30 21:00:00,8.74,286.28,283.40,82.6,226.0,0.0,95690001_729_21
4409472,95690001,2017-12-30 22:00:00,9.04,286.21,283.29,82.4,224.0,0.0,95690001_729_22


Missing Data : 6899007 
 Shape is : (4409474, 9)


In [4]:
#Y_train
ytrain = pd.read_csv(ytrain_path, parse_dates=['date'], infer_datetime_format=True)
ytrain['number_sta'] = ytrain['number_sta'].astype('category')
display(ytrain)
print('Missing Data :', ytrain.isna().sum().sum(), '\n', 'Shape is :', ytrain.shape)

,date,number_sta,Ground_truth,Id
0,2016-01-02,14066001,3.4,14066001_0
1,2016-01-02,14126001,0.5,14126001_0
2,2016-01-02,14137001,3.4,14137001_0
3,2016-01-02,14216001,4.0,14216001_0
4,2016-01-02,14296001,13.3,14296001_0
...,...,...,...,...
183742,2017-12-31,86137003,5.0,86137003_729
183743,2017-12-31,86165005,3.2,86165005_729
183744,2017-12-31,86272002,1.8,86272002_729
183745,2017-12-31,91200002,1.6,91200002_729


Missing Data : 21640 
 Shape is : (183747, 4)


In [5]:
##X_test
xtestS = pd.read_csv(xtestS_path,infer_datetime_format=True)
display(xtestS)
print('Missing Data :', xtestS.isna().sum().sum(), '\n', 'Shape is :', xtestS.shape)

,dd,hu,td,t,ff,precip,month,Id
0,NaN,NaN,NaN,278.35,NaN,NaN,12,14047002_277_4
1,NaN,NaN,NaN,278.40,NaN,0.0,12,14047002_277_5
2,NaN,NaN,NaN,279.01,NaN,0.0,12,14047002_277_6
3,NaN,NaN,NaN,279.66,NaN,0.0,12,14047002_277_7
4,NaN,NaN,NaN,279.99,NaN,0.0,12,14047002_277_8
...,...,...,...,...,...,...,...,...
2304797,190.0,82.8,277.00,279.74,10.62,0.0,12,95690001_176_19
2304798,195.0,84.2,277.44,279.93,11.86,0.0,12,95690001_176_20
2304799,199.0,85.7,277.95,280.21,11.77,0.0,12,95690001_176_21
2304800,198.0,85.3,278.25,280.58,10.16,0.0,12,95690001_176_22


Missing Data : 3914287 
 Shape is : (2304802, 8)


In [6]:
bl_obsT = pd.read_csv(bl_obsT_path,infer_datetime_format=True)
Id_pred = bl_obsT["Id"]

## <font color = 'royalblue'>3 Pré-processing</font>

In [7]:
def Get_StationDay (id):
    return id.split('_')[0] + "_" + id.split('_')[1]

### 3.1 Train data

**Remove NaNs**

In [8]:
xtrainS.dropna(inplace=True)
display(xtrainS.head())
print('Missing Data :', xtrainS.isna().sum().sum(), '\n', 'Shape is :', xtrainS.shape)

,number_sta,date,ff,t,td,hu,dd,precip,Id
0,14066001,2016-01-01 00:00:00,3.05,279.28,277.97,91.4,200.0,0.0,14066001_0_0
1,14066001,2016-01-01 01:00:00,2.57,278.76,277.45,91.4,190.0,0.0,14066001_0_1
2,14066001,2016-01-01 02:00:00,2.26,278.27,277.02,91.7,181.0,0.0,14066001_0_2
3,14066001,2016-01-01 03:00:00,2.62,277.98,276.95,93.0,159.0,0.0,14066001_0_3
4,14066001,2016-01-01 04:00:00,2.99,277.32,276.72,95.9,171.0,0.0,14066001_0_4


Missing Data : 0 
 Shape is : (2264105, 9)


In [9]:
ytrain.dropna(inplace=True)
display(ytrain.head())
print('Missing Data :', ytrain.isna().sum().sum(), '\n', 'Shape is :', ytrain.shape)

,date,number_sta,Ground_truth,Id
0,2016-01-02,14066001,3.4,14066001_0
1,2016-01-02,14126001,0.5,14126001_0
2,2016-01-02,14137001,3.4,14137001_0
3,2016-01-02,14216001,4.0,14216001_0
4,2016-01-02,14296001,13.3,14296001_0


Missing Data : 0 
 Shape is : (162107, 4)


**Create 'StationDay' column**

In [10]:
xtrainS = xtrainS.assign(StationDay=pd.Series(np.zeros(xtrainS.shape[0])).values)
xtrainS.loc[:,"StationDay"] = xtrainS.loc[:,"Id"].apply(Get_StationDay)
display(xtrainS.head())
print('Missing Data :', xtrainS.isna().sum().sum(), '\n', 'Shape is :', xtrainS.shape)

,number_sta,date,ff,t,td,hu,dd,precip,Id,StationDay
0,14066001,2016-01-01 00:00:00,3.05,279.28,277.97,91.4,200.0,0.0,14066001_0_0,14066001_0
1,14066001,2016-01-01 01:00:00,2.57,278.76,277.45,91.4,190.0,0.0,14066001_0_1,14066001_0
2,14066001,2016-01-01 02:00:00,2.26,278.27,277.02,91.7,181.0,0.0,14066001_0_2,14066001_0
3,14066001,2016-01-01 03:00:00,2.62,277.98,276.95,93.0,159.0,0.0,14066001_0_3,14066001_0
4,14066001,2016-01-01 04:00:00,2.99,277.32,276.72,95.9,171.0,0.0,14066001_0_4,14066001_0


Missing Data : 0 
 Shape is : (2264105, 10)


**Drop Useless columns**

In [11]:
xtrainS.drop(['number_sta', 'date', 'Id'], axis=1, inplace=True)
display(xtrainS.head())
print('Missing Data :', xtrainS.isna().sum().sum(), '\n', 'Shape is :', xtrainS.shape)

,ff,t,td,hu,dd,precip,StationDay
0,3.05,279.28,277.97,91.4,200.0,0.0,14066001_0
1,2.57,278.76,277.45,91.4,190.0,0.0,14066001_0
2,2.26,278.27,277.02,91.7,181.0,0.0,14066001_0
3,2.62,277.98,276.95,93.0,159.0,0.0,14066001_0
4,2.99,277.32,276.72,95.9,171.0,0.0,14066001_0


Missing Data : 0 
 Shape is : (2264105, 7)


In [12]:
ytrain.drop(['date', 'number_sta'], axis=1, inplace=True)
display(ytrain.head())
print('Missing Data :', ytrain.isna().sum().sum(), '\n', 'Shape is :', ytrain.shape)

,Ground_truth,Id
0,3.4,14066001_0
1,0.5,14126001_0
2,3.4,14137001_0
3,4.0,14216001_0
4,13.3,14296001_0


Missing Data : 0 
 Shape is : (162107, 2)


**Mean on 24 hours (except 'precip')**

In [13]:
xtrainS = xtrainS.groupby("StationDay", as_index=False).agg(pd.Series.sum, min_count = 24)
xtrainS[['dd', 'hu', 'td', 't', 'ff']] = xtrainS[['dd', 'hu', 'td', 't', 'ff']].divide(24)
display(xtrainS.head())
print('Missing Data :', xtrainS.isna().sum().sum(), '\n', 'Shape is :', xtrainS.shape)

,StationDay,ff,t,td,hu,dd,precip
0,14066001_0,3.913750,280.333750,278.514583,88.591667,146.500000,0.2
1,14066001_1,8.041250,282.936667,279.997500,82.300000,205.625000,3.4
2,14066001_10,5.408750,279.557917,277.497917,86.750000,209.541667,6.0
3,14066001_100,4.296250,282.112917,277.944583,76.408333,134.958333,11.6
4,14066001_101,1.754583,282.805000,281.003333,88.745833,167.208333,5.6


Missing Data : 58296 
 Shape is : (96009, 7)


**Remove NaNs**

In [14]:
xtrainS.dropna(inplace=True)
display(xtrainS.head())
print('Missing Data :', xtrainS.isna().sum().sum(), '\n', 'Shape is :', xtrainS.shape)

,StationDay,ff,t,td,hu,dd,precip
0,14066001_0,3.913750,280.333750,278.514583,88.591667,146.500000,0.2
1,14066001_1,8.041250,282.936667,279.997500,82.300000,205.625000,3.4
2,14066001_10,5.408750,279.557917,277.497917,86.750000,209.541667,6.0
3,14066001_100,4.296250,282.112917,277.944583,76.408333,134.958333,11.6
4,14066001_101,1.754583,282.805000,281.003333,88.745833,167.208333,5.6


Missing Data : 0 
 Shape is : (86293, 7)


**Keep the same rows in X_train and Y_train**

In [15]:
xtrainS = xtrainS.loc[xtrainS['StationDay'].isin(ytrain["Id"])]
display(xtrainS.head())
print('Missing Data :', xtrainS.isna().sum().sum(), '\n', 'Shape is :', xtrainS.shape)

,StationDay,ff,t,td,hu,dd,precip
0,14066001_0,3.913750,280.333750,278.514583,88.591667,146.500000,0.2
1,14066001_1,8.041250,282.936667,279.997500,82.300000,205.625000,3.4
2,14066001_10,5.408750,279.557917,277.497917,86.750000,209.541667,6.0
3,14066001_100,4.296250,282.112917,277.944583,76.408333,134.958333,11.6
4,14066001_101,1.754583,282.805000,281.003333,88.745833,167.208333,5.6


Missing Data : 0 
 Shape is : (82120, 7)


In [16]:
ytrain = ytrain.loc[ytrain['Id'].isin(xtrainS["StationDay"])]
display(ytrain.head())
print('Missing Data :', ytrain.isna().sum().sum(), '\n', 'Shape is :', ytrain.shape)

,Ground_truth,Id
0,3.4,14066001_0
3,4.0,14216001_0
7,10.0,14372001_0
11,3.6,14578001_0
14,10.3,14762004_0


Missing Data : 0 
 Shape is : (82120, 2)


**Reset index**

In [17]:
xtrainS.reset_index(drop=True,inplace=True)
display(xtrainS.head())
print('Missing Data :', xtrainS.isna().sum().sum(), '\n', 'Shape is :', xtrainS.shape)

,StationDay,ff,t,td,hu,dd,precip
0,14066001_0,3.913750,280.333750,278.514583,88.591667,146.500000,0.2
1,14066001_1,8.041250,282.936667,279.997500,82.300000,205.625000,3.4
2,14066001_10,5.408750,279.557917,277.497917,86.750000,209.541667,6.0
3,14066001_100,4.296250,282.112917,277.944583,76.408333,134.958333,11.6
4,14066001_101,1.754583,282.805000,281.003333,88.745833,167.208333,5.6


Missing Data : 0 
 Shape is : (82120, 7)


In [18]:
ytrain.reset_index(drop=True,inplace=True)
display(ytrain.head())
print('Missing Data :', ytrain.isna().sum().sum(), '\n', 'Shape is :', ytrain.shape)

,Ground_truth,Id
0,3.4,14066001_0
1,4.0,14216001_0
2,10.0,14372001_0
3,3.6,14578001_0
4,10.3,14762004_0


Missing Data : 0 
 Shape is : (82120, 2)


**Sort X_train**

In [19]:
xtrainS = xtrainS.set_index('StationDay')
xtrainS = xtrainS.reindex(index=ytrain['Id'])
xtrainS = xtrainS.reset_index()
display(xtrainS.head())
print('Missing Data :', xtrainS.isna().sum().sum(), '\n', 'Shape is :', xtrainS.shape)

,Id,ff,t,td,hu,dd,precip
0,14066001_0,3.913750,280.333750,278.514583,88.591667,146.500000,0.2
1,14216001_0,4.759583,280.089583,278.888750,92.187500,158.666667,0.6
2,14372001_0,5.125833,280.076250,278.564167,90.308333,164.625000,0.2
3,14578001_0,4.868750,279.287500,278.418333,94.266667,167.625000,0.8
4,14762004_0,3.821250,281.128750,278.964583,86.545833,131.500000,0.0


Missing Data : 0 
 Shape is : (82120, 7)


**Drop useless variables**

In [20]:
xtrainS.drop(['Id'], axis=1, inplace=True)
display(xtrainS.head())
print('Missing Data :', xtrainS.isna().sum().sum(), '\n', 'Shape is :', xtrainS.shape)

,ff,t,td,hu,dd,precip
0,3.913750,280.333750,278.514583,88.591667,146.500000,0.2
1,4.759583,280.089583,278.888750,92.187500,158.666667,0.6
2,5.125833,280.076250,278.564167,90.308333,164.625000,0.2
3,4.868750,279.287500,278.418333,94.266667,167.625000,0.8
4,3.821250,281.128750,278.964583,86.545833,131.500000,0.0


Missing Data : 0 
 Shape is : (82120, 6)


In [21]:
ytrain.drop(['Id'], axis=1, inplace=True)
display(ytrain.head())
print('Missing Data :', ytrain.isna().sum().sum(), '\n', 'Shape is :', ytrain.shape)

,Ground_truth
0,3.4
1,4.0
2,10.0
3,3.6
4,10.3


Missing Data : 0 
 Shape is : (82120, 1)


# Normalisation pour réseau de neuronnes

### 3.2 Test data

In [22]:
def Id_to_int (id):
    return int(id.split('_')[0] + id.split('_')[1] + id.split('_')[2])

**Create 'IntId' column**

In [23]:
xtestS = xtestS.assign(IntId=pd.Series(np.zeros(xtestS.shape[0])).values)
xtestS.loc[:,"IntId"] = xtestS.loc[:,"Id"].apply(Id_to_int)
display(xtestS.head())
print('Missing Data :', xtestS.isna().sum().sum(), '\n', 'Shape is :', xtestS.shape)

,dd,hu,td,t,ff,precip,month,Id,IntId
0,NaN,NaN,NaN,278.35,NaN,NaN,12,14047002_277_4,140470022774
1,NaN,NaN,NaN,278.40,NaN,0.0,12,14047002_277_5,140470022775
2,NaN,NaN,NaN,279.01,NaN,0.0,12,14047002_277_6,140470022776
3,NaN,NaN,NaN,279.66,NaN,0.0,12,14047002_277_7,140470022777
4,NaN,NaN,NaN,279.99,NaN,0.0,12,14047002_277_8,140470022778


Missing Data : 3914287 
 Shape is : (2304802, 9)


**Sort X_test on 'IntId' values**

In [24]:
xtestS = xtestS.sort_values('IntId')
display(xtestS.head())
print('Missing Data :', xtestS.isna().sum().sum(), '\n', 'Shape is :', xtestS.shape)

,dd,hu,td,t,ff,precip,month,Id,IntId
1322940,224.0,96.1,277.17,277.75,3.46,0.0,2,14066001_0_0,1406600100
1322941,216.0,96.7,277.47,277.93,3.65,0.0,2,14066001_0_1,1406600101
1322942,222.0,97.1,277.03,277.42,3.63,0.0,2,14066001_0_2,1406600102
1322943,220.0,97.9,277.23,277.54,4.17,0.0,2,14066001_0_3,1406600103
1322944,215.0,98.0,277.30,277.60,4.03,0.0,2,14066001_0_4,1406600104


Missing Data : 3914287 
 Shape is : (2304802, 9)


**Fill NaNs**

In [25]:
xtestS = xtestS.fillna(method="backfill")
display(xtestS.head())
print('Missing Data :', xtestS.isna().sum().sum(), '\n', 'Shape is :', xtestS.shape)

,dd,hu,td,t,ff,precip,month,Id,IntId
1322940,224.0,96.1,277.17,277.75,3.46,0.0,2,14066001_0_0,1406600100
1322941,216.0,96.7,277.47,277.93,3.65,0.0,2,14066001_0_1,1406600101
1322942,222.0,97.1,277.03,277.42,3.63,0.0,2,14066001_0_2,1406600102
1322943,220.0,97.9,277.23,277.54,4.17,0.0,2,14066001_0_3,1406600103
1322944,215.0,98.0,277.30,277.60,4.03,0.0,2,14066001_0_4,1406600104


Missing Data : 0 
 Shape is : (2304802, 9)


**Create 'StationDay' column**

In [26]:
xtestS = xtestS.assign(StationDay=pd.Series(np.zeros(xtestS.shape[0])).values)
xtestS.loc[:,"StationDay"] = xtestS.loc[:,"Id"].apply(Get_StationDay)
display(xtestS.head())
print('Missing Data :', xtestS.isna().sum().sum(), '\n', 'Shape is :', xtestS.shape)

,dd,hu,td,t,ff,precip,month,Id,IntId,StationDay
1322940,224.0,96.1,277.17,277.75,3.46,0.0,2,14066001_0_0,1406600100,14066001_0
1322941,216.0,96.7,277.47,277.93,3.65,0.0,2,14066001_0_1,1406600101,14066001_0
1322942,222.0,97.1,277.03,277.42,3.63,0.0,2,14066001_0_2,1406600102,14066001_0
1322943,220.0,97.9,277.23,277.54,4.17,0.0,2,14066001_0_3,1406600103,14066001_0
1322944,215.0,98.0,277.30,277.60,4.03,0.0,2,14066001_0_4,1406600104,14066001_0


Missing Data : 0 
 Shape is : (2304802, 10)


**Drop useless variables**

In [27]:
xtestS.drop(['month', 'Id', 'IntId'], axis=1, inplace=True)
display(xtestS.head())
print('Missing Data :', xtestS.isna().sum().sum(), '\n', 'Shape is :', xtestS.shape)

,dd,hu,td,t,ff,precip,StationDay
1322940,224.0,96.1,277.17,277.75,3.46,0.0,14066001_0
1322941,216.0,96.7,277.47,277.93,3.65,0.0,14066001_0
1322942,222.0,97.1,277.03,277.42,3.63,0.0,14066001_0
1322943,220.0,97.9,277.23,277.54,4.17,0.0,14066001_0
1322944,215.0,98.0,277.30,277.60,4.03,0.0,14066001_0


Missing Data : 0 
 Shape is : (2304802, 7)


**Mean on 24 hours (except 'precip')**

In [28]:
xtestS = xtestS.groupby("StationDay", as_index=False).agg(pd.Series.sum, min_count = 24)
xtestS[['dd', 'hu', 'td', 't', 'ff']] = xtestS[['dd', 'hu', 'td', 't', 'ff']].divide(24)
display(xtestS.head())
print('Missing Data :', xtestS.isna().sum().sum(), '\n', 'Shape is :', xtestS.shape)

,StationDay,dd,hu,td,t,ff,precip
0,14047002_100,183.583333,90.041667,280.165833,282.565000,3.250833,9.9
1,14047002_223,183.583333,90.041667,280.165833,285.415000,3.250833,0.8
2,14047002_236,183.583333,90.041667,280.165833,283.064167,3.250833,7.0
3,14047002_256,183.583333,90.041667,280.165833,283.611250,3.250833,4.0
4,14047002_277,NaN,NaN,NaN,NaN,NaN,NaN


Missing Data : 258 
 Shape is : (96055, 7)


**Remove NaNs**

In [29]:
xtestS.dropna(inplace=True)
display(xtestS.head())
print('Missing Data :', xtestS.isna().sum().sum(), '\n', 'Shape is :', xtestS.shape)

,StationDay,dd,hu,td,t,ff,precip
0,14047002_100,183.583333,90.041667,280.165833,282.565000,3.250833,9.9
1,14047002_223,183.583333,90.041667,280.165833,285.415000,3.250833,0.8
2,14047002_236,183.583333,90.041667,280.165833,283.064167,3.250833,7.0
3,14047002_256,183.583333,90.041667,280.165833,283.611250,3.250833,4.0
5,14047002_281,183.583333,90.041667,280.165833,282.633750,3.250833,0.0


Missing Data : 0 
 Shape is : (96012, 7)


**Keep the same rows in X_test and Y_test**

In [30]:
xtestS = xtestS.loc[xtestS['StationDay'].isin(bl_obsT["Id"])]
display(xtestS.head())
print('Missing Data :', xtestS.isna().sum().sum(), '\n', 'Shape is :', xtestS.shape)

,StationDay,dd,hu,td,t,ff,precip
0,14047002_100,183.583333,90.041667,280.165833,282.565000,3.250833,9.9
1,14047002_223,183.583333,90.041667,280.165833,285.415000,3.250833,0.8
2,14047002_236,183.583333,90.041667,280.165833,283.064167,3.250833,7.0
3,14047002_256,183.583333,90.041667,280.165833,283.611250,3.250833,4.0
5,14047002_281,183.583333,90.041667,280.165833,282.633750,3.250833,0.0


Missing Data : 0 
 Shape is : (85140, 7)


**Reset index**

In [31]:
xtestS.reset_index(drop=True,inplace=True)
display(xtestS.head())
print('Missing Data :', xtestS.isna().sum().sum(), '\n', 'Shape is :', xtestS.shape)

,StationDay,dd,hu,td,t,ff,precip
0,14047002_100,183.583333,90.041667,280.165833,282.565000,3.250833,9.9
1,14047002_223,183.583333,90.041667,280.165833,285.415000,3.250833,0.8
2,14047002_236,183.583333,90.041667,280.165833,283.064167,3.250833,7.0
3,14047002_256,183.583333,90.041667,280.165833,283.611250,3.250833,4.0
4,14047002_281,183.583333,90.041667,280.165833,282.633750,3.250833,0.0


Missing Data : 0 
 Shape is : (85140, 7)


**Sort X_test**

In [32]:
xtestS = xtestS.set_index('StationDay')
xtestS = xtestS.reindex(index=bl_obsT['Id'])
xtestS = xtestS.reset_index()
display(xtestS.head())
print('Missing Data :', xtestS.isna().sum().sum(), '\n', 'Shape is :', xtestS.shape)

,Id,dd,hu,td,t,ff,precip
0,14066001_149,224.625000,81.720833,279.107917,282.099583,5.237917,1.4
1,14126001_149,163.166667,92.291667,279.718750,280.895833,4.103333,1.0
2,14137001_149,218.458333,83.266667,278.469583,281.167917,5.547083,0.4
3,14216001_149,207.375000,83.345833,278.105417,280.804583,2.667083,0.8
4,14296001_149,191.250000,63.250000,272.062500,279.570417,3.254167,3.6


Missing Data : 0 
 Shape is : (85140, 7)


**Drop useless variables**

In [33]:
xtestS.drop(['Id'], axis=1, inplace=True)
display(xtestS.head())
print('Missing Data :', xtestS.isna().sum().sum(), '\n', 'Shape is :', xtestS.shape)

,dd,hu,td,t,ff,precip
0,224.625000,81.720833,279.107917,282.099583,5.237917,1.4
1,163.166667,92.291667,279.718750,280.895833,4.103333,1.0
2,218.458333,83.266667,278.469583,281.167917,5.547083,0.4
3,207.375000,83.345833,278.105417,280.804583,2.667083,0.8
4,191.250000,63.250000,272.062500,279.570417,3.254167,3.6


Missing Data : 0 
 Shape is : (85140, 6)


**Rearrange columns order**

In [34]:
xtestS = xtestS[['ff', 't', 'td', 'hu', 'dd', 'precip']]
display(xtestS.head())
print('Missing Data :', xtestS.isna().sum().sum(), '\n', 'Shape is :', xtestS.shape)

,ff,t,td,hu,dd,precip
0,5.237917,282.099583,279.107917,81.720833,224.625000,1.4
1,4.103333,280.895833,279.718750,92.291667,163.166667,1.0
2,5.547083,281.167917,278.469583,83.266667,218.458333,0.4
3,2.667083,280.804583,278.105417,83.345833,207.375000,0.8
4,3.254167,279.570417,272.062500,63.250000,191.250000,3.6


Missing Data : 0 
 Shape is : (85140, 6)


## <font color = 'royalblue'>4 Apprentissage</font>

### 4.1 Optimal tree

In [35]:
# Optimisation de la profondeur de l'arbre
param = [{"max_depth":list(range(2,10))}]
tree = GridSearchCV(DecisionTreeRegressor(), param, cv=10, n_jobs=-1)

ts = time.time()
treeOpt = tree.fit(xtrainS, ytrain)
# paramètre optimal
print("Meilleur score = %f, Meilleur paramètre = %s" % (treeOpt.best_score_,treeOpt.best_params_))
te = time.time()

print("Time running = %f" % (te-ts))

Meilleur score = 0.047601, Meilleur paramètre = {'max_depth': 3}
Time running = 16.829085


### 4.2 Random forest

In [44]:
#Optimisation des paramètres de la forêt
param=[{"max_features":list(range(2,10,1))}]
rf= GridSearchCV(RandomForestRegressor(n_estimators=100), param, cv=5, n_jobs=-1)

ts = time.time()
rfOpt = rf.fit(xtrainS, ytrain.values.ravel())
# paramètre optimal
print("Meilleur score = %f, Meilleur paramètre = %s" % (rfOpt.best_score_,rfOpt.best_params_))
te = time.time()

print("Time running = %f" % (te-ts))

Meilleur score = -0.008771, Meilleur paramètre = {'max_features': 2}
Time running = 416.334420


### 4.3 Neural network

In [56]:
xtrainS_N = StandardScaler().fit_transform(xtrainS)
xtestS_N = ((xtestS - xtrainS.mean())/xtrainS.std()).to_numpy()

In [58]:
#Optimisation des paramètres du réseau
param_grid=[{"hidden_layer_sizes":list([(5,),(6,),(7,),(8,)])}]
nnet= GridSearchCV(MLPRegressor(max_iter=5000), param_grid, cv=10, n_jobs=-1)

ts=time.time()
nnetOpt=nnet.fit(xtrainS_N, ytrain.values.ravel())
# paramètre optimal
print("Meilleur score = %f, Meilleur paramètre = %s" % (nnetOpt.best_score_,nnetOpt.best_params_))
te=time.time()

print("Time running = %f" % (te-ts))

Meilleur score = 0.069605, Meilleur paramètre = {'hidden_layer_sizes': (6,)}
Time running = 744.990460


## <font color = 'royalblue'>5 Prédictions</font>

### 5.1 Optimal tree

In [46]:
predictions_tree = treeOpt.predict(xtestS)
Tree_predictions = bl_obsT.copy()
Tree_predictions["Prediction"] = predictions_tree
display(Tree_predictions.head())

,Id,Prediction
0,14066001_149,3.465697
1,14126001_149,3.465697
2,14137001_149,2.064466
3,14216001_149,1.269558
4,14296001_149,3.465697


### 5.2 Random forest

In [45]:
predictions_forest = rfOpt.predict(xtestS)
Forest_predictions = bl_obsT.copy()
Forest_predictions["Prediction"] = predictions_forest
display(Forest_predictions.head())

,Id,Prediction
0,14066001_149,7.055
1,14126001_149,3.024
2,14137001_149,7.950
3,14216001_149,3.894
4,14296001_149,2.219


### 5.3 Neural network

In [59]:
predictions_network = nnetOpt.predict(xtestS_N)
Network_predictions = bl_obsT.copy()
Network_predictions["Prediction"] = predictions_network
display(Network_predictions.head())

,Id,Prediction
0,14066001_149,4.795849
1,14126001_149,2.704789
2,14137001_149,4.111453
3,14216001_149,2.776425
4,14296001_149,5.515553


In [60]:
print(np.mean(Network_predictions["Prediction"]))
print(np.mean(Forest_predictions["Prediction"]))
print(np.mean(Tree_predictions["Prediction"]))
print(np.mean(bl_obsT["Prediction"]))

1.9149202738086055
2.7995313601127365
2.146050980901067
2.2113436692506063


## <font color = 'royalblue'>6 Post-processing</font>

### 6.1 Optimal tree

In [62]:
Tree_predictions["Prediction"] = Tree_predictions["Prediction"] + 1
display(Tree_predictions.head())
print('Missing Data :', Tree_predictions.isna().sum().sum(), '\n', 'Shape is :', Tree_predictions.shape)

,Id,Prediction
0,14066001_149,4.465697
1,14126001_149,4.465697
2,14137001_149,3.064466
3,14216001_149,2.269558
4,14296001_149,4.465697


Missing Data : 0 
 Shape is : (85140, 2)


### 6.2 Random forest

In [63]:
Forest_predictions["Prediction"] = Forest_predictions["Prediction"] + 1
display(Forest_predictions.head())
print('Missing Data :', Forest_predictions.isna().sum().sum(), '\n', 'Shape is :', Forest_predictions.shape)

,Id,Prediction
0,14066001_149,8.055
1,14126001_149,4.024
2,14137001_149,8.950
3,14216001_149,4.894
4,14296001_149,3.219


Missing Data : 0 
 Shape is : (85140, 2)


### 6.3 Neural network

In [64]:
Network_predictions["Prediction"] = Network_predictions["Prediction"] + 1
display(Network_predictions.head())
print('Missing Data :', Network_predictions.isna().sum().sum(), '\n', 'Shape is :', Network_predictions.shape)

,Id,Prediction
0,14066001_149,5.795849
1,14126001_149,3.704789
2,14137001_149,5.111453
3,14216001_149,3.776425
4,14296001_149,6.515553


Missing Data : 0 
 Shape is : (85140, 2)


## <font color = 'royalblue'>7 Exportation</font>



In [65]:
output_file = "Submissions_5_neural-network.csv"
Network_predictions.to_csv('/Users/qdouzery/Desktop/INSAT_Defi-IA-2022/Submissions/' + output_file, index=False)

## 8 Autres

In [ ]:
# Variables explicatives : on gère "mode" et "key"
spotifyDum=pd.get_dummies(spotify[['mode','key']])
del spotifyDum['mode_0']
del spotifyDum['key_A']
spotifyDum.head()

In [ ]:
#Normalisation des données
Xtr = StandardScaler().fit_transform(X_train)
Xte = ((X_test - X_train.mean())/X_train.std()).to_numpy()

In [ ]:
##Gérer les outliers